# EECS 738 Comment Toxicity Prediction Model

In [1]:
import sys
sys.path.insert(0, '../src/')
import tools

import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
# establish file toolkit
t = tools.tools('../data/train.csv')
df = t.data

Processing training data...
Removing Punctuation...
Building Stop Word Dictionary...
About to get all good comments


../src\tools.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  comments_scores['comment_text'] = self.processCommentList(comments_scores['comment_text'])


In [3]:
import CommentPredictor as CP

In [4]:
mod = CP.CommentPredictor(t)


In [7]:
preds = mod.getPreds()

In [6]:
preds.head(5)

,target,comment_text
11,0.00,this is a great story man i wonder if the pe...
31,0.66,yet call out all muslims for the acts of a few...
51,0.00,because the people who drive cars more are the...
58,0.00,mormons have had a complicated relationship wi...
111,0.66,i'm doing the same thing


In [7]:
comms = t.comments_scores

In [8]:
comms.head(5)

,target,comment_text
11,0.4400,this is a great story man i wonder if the pe...
31,0.9125,yet call out all muslims for the acts of a few...
51,0.0000,because the people who drive cars more are the...
58,0.3000,mormons have had a complicated relationship wi...
111,0.0000,i'm doing the same thing


Here we prep the text corpus for neural network training. getProcessedComments gives us all comments, lowercased with punctuation and Unicode characters stripped. We then replace each word in each comment with the calculated preliminary toxicity weight we have calculated for that word. We pad/truncate comments so that each comment is a 200-word sequence, and create a list of target toxicity labels corresponding to these comments. These two lists (input and output) will form the training set for our network.

In [10]:
procComms = (t.getProcessedComments())
labels = df['target']
from keras.preprocessing.text import text_to_word_sequence
wordDict = mod.word_weight_dict
print("Training set size: " + str(len(procComms)))
for index, comment in enumerate(procComms):
    split = text_to_word_sequence(comment)
    for j, word in enumerate(split):
        split[j] = wordDict[word]
    procComms[index] = split
uniqueWords = len(wordDict)
commentLen = 200
X_t = pad_sequences(procComms, maxlen=commentLen)
y = labels
print("Unique words: " + str(uniqueWords))


Removing Punctuation...
Training set size: 235087
Unique words: 124213


In [7]:
net = mod.buildNeuralNet(uniqueWords, commentLen)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                10050     
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 10,101
Trainable params: 10,101
Non-trainable params: 0
_________________________________________________________________


In [8]:
net.fit(X_t, y, batch_size=32, epochs=2, validation_split=.1)

Instructions for updating:
Use tf.cast instead.
Train on 211578 samples, validate on 23509 samples
Epoch 1/2
211578/211578 [==============================] - 24s 114us/step - loss: 0.4267 - acc: 0.6150 - val_loss: 0.4100 - val_acc: 0.5990
Epoch 2/2
211578/211578 [==============================] - 25s 117us/step - loss: 0.3941 - acc: 0.6150 - val_loss: 0.4100 - val_acc: 0.5990
